In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Mesothelioma"
cohort = "GSE107754"

# Input paths
in_trait_dir = "../../input/GEO/Mesothelioma"
in_cohort_dir = "../../input/GEO/Mesothelioma/GSE107754"

# Output paths
out_data_file = "../../output/preprocess/Mesothelioma/GSE107754.csv"
out_gene_data_file = "../../output/preprocess/Mesothelioma/gene_data/GSE107754.csv"
out_clinical_data_file = "../../output/preprocess/Mesothelioma/clinical_data/GSE107754.csv"
json_path = "../../output/preprocess/Mesothelioma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
# 1. Check what files are actually in the directory
import os
print("Files in the directory:")
files = os.listdir(in_cohort_dir)
print(files)

# 2. Find appropriate files with more flexible pattern matching
soft_file = None
matrix_file = None

for file in files:
    file_path = os.path.join(in_cohort_dir, file)
    # Look for files that might contain SOFT or matrix data with various possible extensions
    if 'soft' in file.lower() or 'family' in file.lower() or file.endswith('.soft.gz'):
        soft_file = file_path
    if 'matrix' in file.lower() or file.endswith('.txt.gz') or file.endswith('.tsv.gz'):
        matrix_file = file_path

if not soft_file:
    print("Warning: Could not find a SOFT file. Using the first .gz file as fallback.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if gz_files:
        soft_file = os.path.join(in_cohort_dir, gz_files[0])

if not matrix_file:
    print("Warning: Could not find a matrix file. Using the second .gz file as fallback if available.")
    gz_files = [f for f in files if f.endswith('.gz')]
    if len(gz_files) > 1 and soft_file != os.path.join(in_cohort_dir, gz_files[1]):
        matrix_file = os.path.join(in_cohort_dir, gz_files[1])
    elif len(gz_files) == 1 and not soft_file:
        matrix_file = os.path.join(in_cohort_dir, gz_files[0])

print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# 3. Read files if found
if soft_file and matrix_file:
    # Read the matrix file to obtain background information and sample characteristics data
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    
    try:
        background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
        
        # Obtain the sample characteristics dictionary from the clinical dataframe
        sample_characteristics_dict = get_unique_values_by_row(clinical_data)
        
        # Explicitly print out all the background information and the sample characteristics dictionary
        print("Background Information:")
        print(background_info)
        print("Sample Characteristics Dictionary:")
        print(sample_characteristics_dict)
    except Exception as e:
        print(f"Error processing files: {e}")
        # Try swapping files if first attempt fails
        print("Trying to swap SOFT and matrix files...")
        temp = soft_file
        soft_file = matrix_file
        matrix_file = temp
        try:
            background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
            sample_characteristics_dict = get_unique_values_by_row(clinical_data)
            print("Background Information:")
            print(background_info)
            print("Sample Characteristics Dictionary:")
            print(sample_characteristics_dict)
        except Exception as e:
            print(f"Still error after swapping: {e}")
else:
    print("Could not find necessary files for processing.")


Files in the directory:
['GSE107754_family.soft.gz', 'GSE107754_series_matrix.txt.gz']
SOFT file: ../../input/GEO/Mesothelioma/GSE107754/GSE107754_family.soft.gz
Matrix file: ../../input/GEO/Mesothelioma/GSE107754/GSE107754_series_matrix.txt.gz


Background Information:
!Series_title	"A novel genomic signature predicting FDG uptake in diverse metastatic tumors"
!Series_summary	"Purpose: Building a universal genomic signature predicting the intensity of FDG uptake in diverse metastatic tumors may allow us to understand better the biological processes underlying this phenomenon and their requirements of glucose uptake."
!Series_summary	"Methods: A balanced training set (n=71) of metastatic tumors including some of the most frequent histologies, with matched PET/CT quantification measurements and whole human genome gene expression microarrays, was used to build the signature. Selection of microarray features was carried out exclusively on the basis of their strong association with FDG uptake (as measured by SUVmean35) by means of univariate linear regression. A thorough bioinformatics study of these genes was performed and multivariable models were built by fitting several state of the art regression techniques to the training set

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
import gzip
from typing import Optional, Callable, Dict, Any
import re

# 1. Gene Expression Data Availability
# Based on the information provided, this dataset appears to contain gene expression data
# as mentioned in the series summary: "whole human genome gene expression microarrays"
is_gene_available = True

# 2.1 Data Availability
# For trait (Mesothelioma), looking at row 2 which contains tissue types
# We can see "tissue: Malignant Mesothelioma" is available
trait_row = 2

# Age is not available in the sample characteristics
age_row = None

# Gender is available in row 0
gender_row = 0

# 2.2 Data Type Conversion
def convert_trait(value: str) -> Optional[int]:
    """
    Convert tissue type to binary: 1 if Mesothelioma, 0 otherwise
    """
    if value is None:
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Check if it's Mesothelioma (case-insensitive)
    if re.search(r'malignant\s+mesothelioma', value, re.IGNORECASE):
        return 1
    else:
        return 0

def convert_age(value: str) -> Optional[float]:
    """
    Convert age to float. Not used in this dataset as age is not available.
    """
    if value is None:
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value: str) -> Optional[int]:
    """
    Convert gender to binary: 0 for female, 1 for male
    """
    if value is None:
        return None
    
    # Extract the value after the colon
    if ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if value == "male":
        return 1
    elif value == "female":
        return 0
    else:
        return None

# 3. Save Metadata
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None

# Conduct initial filtering on the usability of the dataset
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Parse the series matrix file to extract clinical data
    matrix_file_path = os.path.join(in_cohort_dir, "GSE107754_series_matrix.txt.gz")
    
    samples = []
    characteristics = []
    
    with gzip.open(matrix_file_path, 'rt') as f:
        for line in f:
            if line.startswith('!Sample_geo_accession'):
                samples = line.strip().split('\t')[1:]
            elif line.startswith('!Sample_characteristics_ch1'):
                characteristics.append(line.strip().split('\t')[1:])
            # Stop once we've passed the characteristics section
            elif samples and characteristics and not line.startswith('!Sample_'):
                break
    
    # Create a DataFrame with samples as columns and characteristics as rows
    if samples and characteristics:
        clinical_data = pd.DataFrame(characteristics, columns=samples)
        
        # Extract clinical features
        clinical_features = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the extracted clinical features
        preview = preview_df(clinical_features)
        print("Preview of clinical features:", preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save the clinical features to CSV
        clinical_features.to_csv(out_clinical_data_file)
        print(f"Clinical features saved to {out_clinical_data_file}")
    else:
        print("Could not extract sample information or characteristics from the matrix file.")


Preview of clinical features: {'"GSM2878070"': [0.0, nan], '"GSM2878071"': [0.0, nan], '"GSM2878072"': [0.0, nan], '"GSM2878073"': [0.0, nan], '"GSM2878074"': [0.0, nan], '"GSM2878075"': [0.0, nan], '"GSM2878076"': [0.0, nan], '"GSM2878077"': [0.0, nan], '"GSM2878078"': [0.0, nan], '"GSM2878079"': [0.0, nan], '"GSM2878080"': [0.0, nan], '"GSM2878081"': [0.0, nan], '"GSM2878082"': [0.0, nan], '"GSM2891194"': [0.0, nan], '"GSM2891195"': [0.0, nan], '"GSM2891196"': [0.0, nan], '"GSM2891197"': [0.0, nan], '"GSM2891198"': [0.0, nan], '"GSM2891199"': [0.0, nan], '"GSM2891200"': [0.0, nan], '"GSM2891201"': [0.0, nan], '"GSM2891202"': [0.0, nan], '"GSM2891203"': [0.0, nan], '"GSM2891204"': [0.0, nan], '"GSM2891205"': [0.0, nan], '"GSM2891206"': [0.0, nan], '"GSM2891207"': [0.0, nan], '"GSM2891208"': [0.0, nan], '"GSM2891209"': [0.0, nan], '"GSM2891210"': [0.0, nan], '"GSM2891211"': [0.0, nan], '"GSM2891212"': [0.0, nan], '"GSM2891213"': [0.0, nan], '"GSM2891214"': [0.0, nan], '"GSM2891215"': [

### Step 3: Gene Data Extraction

In [4]:
# 1. First get the path to the soft and matrix files
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Looking more carefully at the background information
# This is a SuperSeries which doesn't contain direct gene expression data
# Need to investigate the soft file to find the subseries
print("This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:")

# Open the SOFT file to try to identify subseries
with gzip.open(soft_file, 'rt') as f:
    subseries_lines = []
    for i, line in enumerate(f):
        if 'Series_relation' in line and 'SuperSeries of' in line:
            subseries_lines.append(line.strip())
        if i > 1000:  # Limit search to first 1000 lines
            break

# Display the subseries found
if subseries_lines:
    print("Found potential subseries references:")
    for line in subseries_lines:
        print(line)
else:
    print("No subseries references found in the first 1000 lines of the SOFT file.")

# Despite trying to extract gene data, we expect it might fail because this is a SuperSeries
try:
    gene_data = get_genetic_data(matrix_file)
    print("\nGene data extraction result:")
    print("Number of rows:", len(gene_data))
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This confirms the dataset is a SuperSeries without direct gene expression data.")


This appears to be a SuperSeries. Looking at the SOFT file to find potential subseries:
No subseries references found in the first 1000 lines of the SOFT file.



Gene data extraction result:
Number of rows: 41000
First 20 gene/probe identifiers:
Index(['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056',
       'A_23_P100074', 'A_23_P100092', 'A_23_P100103', 'A_23_P100111',
       'A_23_P100127', 'A_23_P100133', 'A_23_P100141', 'A_23_P100156',
       'A_23_P100177', 'A_23_P100189', 'A_23_P100196', 'A_23_P100203',
       'A_23_P100220', 'A_23_P100240', 'A_23_P10025', 'A_23_P100263'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers from the previous step
# These identifiers like 'A_23_P100001' appear to be Agilent microarray probe IDs, not human gene symbols
# These need to be mapped to official gene symbols for biological interpretation

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'SPOT_ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'CONTROL_TYPE': ['FALSE', 'FALSE', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GB_ACC': ['NM_207446', 'NM_005829', 'NM_014848', 'NM_194272', 'NM_020371'], 'GENE': [400451.0, 10239.0, 9899.0, 348093.0, 57099.0], 'GENE_SYMBOL': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN'], 'GENE_NAME': ['family with sequence similarity 174, member B', 'adaptor-related protein complex 3, sigma 2 subunit', 'synaptic vesicle glycoprotein 2B', 'RNA binding protein with multiple splicing 2', 'apoptosis, caspase activation inhibitor'], 'UNIGENE_ID': ['Hs.27373', 'Hs.632161', 'Hs.21754', 'Hs.436518', 'Hs.555966'], 'ENSEMBL_ID': ['ENST00000557398', nan, 'ENST00000557410', 'ENST00000300069', 'ENST00000306730'], 'TIGR_ID': [nan, nan, nan, nan, nan]

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Observing the gene identifiers in the gene expression data:
# From step 3, we see identifiers like 'A_23_P100001' 
# From step 5, we see the same identifiers in the 'ID' column of gene_annotation dataframe
# The gene symbols are stored in the 'GENE_SYMBOL' column

# 2. Get a gene mapping dataframe
probe_col = 'ID'  # Column with probe identifiers
gene_col = 'GENE_SYMBOL'  # Column with gene symbols
gene_mapping = get_gene_mapping(gene_annotation, probe_col, gene_col)

print("Gene mapping preview:")
print(preview_df(gene_mapping))

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

print("\nGene expression data after mapping:")
print("Number of genes:", len(gene_data))
print("First 5 gene symbols:")
print(gene_data.index[:5])

# Save gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping preview:
{'ID': ['A_23_P100001', 'A_23_P100011', 'A_23_P100022', 'A_23_P100056', 'A_23_P100074'], 'Gene': ['FAM174B', 'AP3S2', 'SV2B', 'RBPMS2', 'AVEN']}



Gene expression data after mapping:
Number of genes: 18488
First 5 gene symbols:
Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2LD1', 'A2M'], dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Mesothelioma/gene_data/GSE107754.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {gene_data_normalized.shape}")
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data_normalized.to_csv(out_gene_data_file)
print(f"Saved normalized gene data to {out_gene_data_file}")

# 2. Create clinical data from the sample IDs in the gene expression data
# Since all samples are from tumor tissue, they all have mesothelioma (but this is not useful as a trait)
sample_ids = gene_data.columns.tolist()
print(f"Number of samples: {len(sample_ids)}")

# Extract gender information from the original matrix file
gender_data = {}
try:
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    
    # Read the matrix file to extract sample characteristics
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Display unique values in each row of clinical data
    characteristics_dict = get_unique_values_by_row(clinical_data)
    print("Sample characteristics dictionary:")
    print(characteristics_dict)
    
    # Try to find gender information in the characteristics
    gender_row = None
    for idx, values in characteristics_dict.items():
        if any('sex:' in str(v).lower() for v in values):
            gender_row = idx
            break
    
    if gender_row is not None:
        # Extract gender data from the row
        for col in clinical_data.columns:
            if col != '!Sample_geo_accession':
                continue
                
            for idx, row in clinical_data.iterrows():
                if idx == gender_row:
                    for i, sample_id in enumerate(clinical_data.iloc[0].values):
                        if i > 0 and sample_id in sample_ids:  # Skip the first column (header)
                            gender_val = clinical_data.iloc[idx, i]
                            if isinstance(gender_val, str) and 'sex:' in gender_val.lower():
                                gender = 0 if 'f' in gender_val.lower() else 1 if 'm' in gender_val.lower() else None
                                gender_data[sample_id] = gender
except Exception as e:
    print(f"Error extracting gender data: {e}")

# Create a clinical dataframe
clinical_df = pd.DataFrame(index=sample_ids)
clinical_df['Mesothelioma'] = 1  # All samples have mesothelioma

# Add gender if available
if gender_data:
    clinical_df['Gender'] = clinical_df.index.map(lambda x: gender_data.get(x))
    print(f"Added gender data for {sum(pd.notna(clinical_df['Gender']))} samples")

print("Clinical data preview:")
print(clinical_df.head())

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_df.to_csv(out_clinical_data_file)
print(f"Saved clinical data to {out_clinical_data_file}")

# 3. Link clinical and genetic data (transpose gene expression data to have samples as rows)
linked_data = pd.concat([clinical_df, gene_data_normalized.T], axis=1)
print(f"Shape of linked data: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data_cleaned = handle_missing_values(linked_data, 'Mesothelioma')
print(f"Shape of linked data after handling missing values: {linked_data_cleaned.shape}")

# 5. Check if the trait is biased (it will be since all samples are mesothelioma)
is_trait_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data_cleaned, 'Mesothelioma')

# 6. Validate the dataset and save cohort information
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,  # We do have trait data, it's just that all values are the same
    is_biased=is_trait_biased,  # This will be True since all samples have the same trait value
    df=unbiased_linked_data,
    note="Dataset contains gene expression data from mesothelioma patients only (no controls), making trait biased."
)

# 7. Save the linked data if it's usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Saved processed linked data to {out_data_file}")
else:
    print("Dataset validation failed due to biased trait variable. Final linked data not saved.")

Shape of gene data after normalization: (18247, 84)


Saved normalized gene data to ../../output/preprocess/Mesothelioma/gene_data/GSE107754.csv
Number of samples: 84
Sample characteristics dictionary:
{0: ['gender: Male', 'gender: Female'], 1: ['dataset: Validation set', 'dataset: Training set'], 2: ['biopsy location: Lung', 'biopsy location: Lymph node', 'biopsy location: Primary', 'biopsy location: Liver', 'biopsy location: Retroperitoneal implant', 'tissue: Pancreatic cancer', 'tissue: Esophagus cancer', 'tissue: Breast cancer', 'tissue: Colorectal cancer', 'tissue: Ovarian cancer', 'tissue: Head&neck cancer', 'tissue: Lung cancer', 'tissue: Malignant Melanoma', 'tissue: Endometrial cancer', 'tissue: Cervix cancer', 'tissue: Soft tissue sarcoma', 'tissue: Gastric cancer', 'tissue: Unknown primary', 'tissue: Malignant Mesothelioma', 'tissue: Thyroid cancer', 'tissue: Testes cancer', 'tissue: Non Hodgkin lymphoma', 'tissue: Merkel cell carcinoma', 'tissue: Vaginal cancer', 'tissue: Kidney cancer', 'tissue: Cervical cancer', 'tissue: Bil

            Mesothelioma
GSM2878070             1
GSM2878071             1
GSM2878072             1
GSM2878073             1
GSM2878074             1
Saved clinical data to ../../output/preprocess/Mesothelioma/clinical_data/GSE107754.csv
Shape of linked data: (84, 18248)


Shape of linked data after handling missing values: (84, 18248)
Quartiles for 'Mesothelioma':
  25%: 1.0
  50% (Median): 1.0
  75%: 1.0
Min: 1
Max: 1
The distribution of the feature 'Mesothelioma' in this dataset is severely biased.

A new JSON file was created at: ../../output/preprocess/Mesothelioma/cohort_info.json
Dataset validation failed due to biased trait variable. Final linked data not saved.
